In [4]:
import numpy as np
from scipy.stats import beta, bernoulli

# Input data
a = 1
b = 1
c = 1  
d = 1   
e = 1
f = 1
epsilon = 0.1

# Initialization
conversions_a = 0
visits_a = 0
conversions_b = 0 
visits_b = 0
conversions_c = 0
visits_c = 0

a_posterior = a
b_posterior = b
c_posterior = c
d_posterior = d
e_posterior = e
f_posterior = f

test_finished = False

expected_loss_a = 0
expected_loss_b = 0
expected_loss_c = 0

page_number_with_min_EL = ''

# Visitor loop
for i in range(10000):
    # Group selection
    group = np.random.choice([1, 2, 3], p=[1/3, 1/3, 1/3])  

    # Behavioral modeling
    if group == 1:
        conversion = bernoulli.rvs(beta.rvs(a_posterior, b_posterior))
        conversions_a += conversion
        visits_a += 1
    elif group == 2:
        conversion = bernoulli.rvs(beta.rvs(c_posterior, d_posterior))
        conversions_b += conversion
        visits_b += 1
    else:
        conversion = bernoulli.rvs(beta.rvs(e_posterior, f_posterior))  
        conversions_c += conversion
        visits_c += 1

    # Parameter update
    a_posterior = a + conversions_a
    b_posterior = b + visits_a - conversions_a
    c_posterior = c + conversions_b 
    d_posterior = d + visits_b - conversions_b
    e_posterior = e + conversions_c
    f_posterior = f + visits_c - conversions_c

    # Expected loss calculation
    expected_loss_a = (
        beta.mean(a_posterior, b_posterior) - 
        beta.mean(c_posterior, d_posterior) *  
        beta.mean(e_posterior, f_posterior)
    )

    expected_loss_b = (
        beta.mean(c_posterior, d_posterior) -
        beta.mean(a_posterior, b_posterior) *
        beta.mean(e_posterior, f_posterior)
    )

    expected_loss_c = (
        beta.mean(e_posterior, f_posterior) - 
        beta.mean(a_posterior, b_posterior) *
        beta.mean(c_posterior, d_posterior)
    )

    # Stopping condition check
    if abs(min(expected_loss_a, expected_loss_b, expected_loss_c) - max(expected_loss_a, expected_loss_b, expected_loss_c)) > epsilon:  
        test_finished = True
        break

# Determine the best option
if expected_loss_a < expected_loss_b and expected_loss_a < expected_loss_c:
    page_number_with_min_EL = 'A' 
elif expected_loss_b < expected_loss_c:
    page_number_with_min_EL = 'B'
else:
    page_number_with_min_EL = 'C'

# Output the result  
print('Optimal option:', page_number_with_min_EL)
